In [1]:
# pip install pdfplumber pandas xlsxwriter openpyxl

import os
import re
import pdfplumber

pdf_path  = r"C:/Users/Famille/OneDrive/Documents/Palmares/2023/Palmares_12_KINSHASA-FUNA.pdf"
xlsx_path = r"C:/Users/Famille/OneDrive/Documents/Palmares/2023/eleves_KINSHASA_FUNA.xlsx"

# Options conservées (on ne change pas ce comportement)
options_map = {
    "101": "LATIN-PHILOSOPHIE",
    "102": "SCIENCES",
    "201": "PEDAGOGIE GENERALE",
    "301": "COMMERCIALE ET GESTION",
}
OPTIONS_ALLOWED = set(options_map.keys())

# --- Normalisation du nom d'école ---
def normalize_school_name(name: str) -> str:
    """
      Règles demandées (ordre important) :
      I. -> INSTITUT ; INST -> INSTITUT ; I.TITUT -> INSTITUT ; I. SC -> INSTITUT SCIENTIFIQUE
      CS / C.S / C. S -> COMPLEXE SCOLAIRE ; CSFRANCO -> COMPLEXE SCOLAIRE FRANCOPHONE
      COL -> COLLEGE ; COLL. -> COLLEGE ; CM -> COLLEGE MODERNE ; COL M. -> COLLEGE MARIE
      L. -> LYCEE ; L. T. -> LYCEE TECHNIQUE ; L. M. -> LYCEE MADAME
      E. -> ECOLE ; ST -> SAINT ; STE -> SAINTE ; ND -> NOTRE DAME ; M. -> MARIE ; JEAN-M. -> JEAN-MARIE
      Rev. -> REVEREND ; Pst -> PASTEUR ; PED -> PEDAGOGIQUE ; D'EX -> D'EXCELLENCE ; Prof -> PROFESSEUR ;  COL SŒURS DE M. ET JESUS -> COLLEGE SOEURS DE MARIE ET JESUS ;
      
    """
    n = (name or "").upper()

    # ordre important (composés avant simples)
    rules = [
        (r'(?<!\w)L\.\s*T\.?(?!\w)', 'LYCEE TECHNIQUE'),         # L. T.
        (r'(?<!\w)I\.\s*T\.?(?!\w)', 'INSTITUT TECHNIQUE'),      # I. T.
        (r'(?<!\w)REV\.?(?!\w)',       'REVEREND'),               # Rev / Rev.
        (r'(?<!\w)C[.\s]*S[.\s]?(?!\w)','COMPLEXE SCOLAIRE'),     # C.S / C. S / CS / C S
        (r'(?<!\w)CSFRANCO(?!\w)',     'COMPLEXE SCOLAIRE FRANCOPHONE'),
        (r'(?<!\w)I\.(?!\w)',         'INSTITUT'),                # I.
        (r'(?<!\w)INST(?!\w)',        'INSTITUT'),                # INST
        (r'(?<!\w)I\.?TITUT(?!\w)',   'INSTITUT'),                # I.TITUT
        (r'(?<!\w)I\.\s*SC(?!\w)',    'INSTITUT SCIENTIFIQUE'),   # I. SC
        (r'(?<!\w)GS(?!\w)',          'GROUPE SCOLAIRE'),         # GS
        (r'(?<!\w)COL(?!\w)',         'COLLEGE'),                 # COL
        (r'(?<!\w)COLL.(?!\w)',       'COLLEGE'),
        (r'(?<!\w)CM(?!\w)',          'COLLEGE MODERNE'),         # CM
        (r'(?<!\w)L\.(?!\w)',         'LYCEE'),
        (r'(?<!\w)L\.\s*M\.(?!\w)',   'LYCEE MADAME'),            # L. M.
        (r'(?<!\w)E\.(?!\w)',         'ECOLE'),                   # E.
        (r'(?<!\w)ST(?!\w)',          'SAINT'),                   # ST
        (r'(?<!\w)STE(?!\w)',         'SAINTE'),                  # STE
        (r'(?<!\w)ND(?!\w)',          'NOTRE DAME'),              # ND
        (r'(?<!\w)M\.(?!\w)',         'MARIE'),                   # M.
        (r'(?<!\w)JEAN-M\.(?!\w)',    'JEAN-MARIE'),
        (r'(?<!\w)COL\s*M\.(?!\w)',   'COLLEGE MARIE'),
        (r'(?<!\w)Pst(?!\w)',         'PASTEUR'),
        (r'(?<!\w)PED(?!\w)',         'PEDAGOGIQUE'),
        (r'(?<!\w)D\'EX(?!\w)',       'D\'EXCELLENCE'),           # D'EX
        (r'(?<!\w)Prof(?!\w)',        'PROFESSEUR'),
    ]
    for pat, rep in rules:
        n = re.sub(pat, rep, n)

    # espaces multiples -> un seul
    n = re.sub(r'\s+', ' ', n).strip()
    return n

# Détection d'un vrai nom d'école (en excluant les entêtes)
ecole_pattern = re.compile(r'^[A-Z0-9\.\s/\-]+$')
def is_header_like(line: str) -> bool:
    l = line.upper().strip()
    return l.startswith(("OPTION", "PROVINCE", "CODE", "PARTICIP", "RÉUSSIT", "REUSSIT"))

percent_re = re.compile(r'^\d+(?:[.,]\d+)?$')
code_ecole_re = re.compile(r'Code\s*:\s*([\d\s/]+)')

def extract_lines(page, bbox):
    """Regroupe les mots en lignes (haut->bas, gauche->droite) dans une bbox."""
    words = page.crop(bbox).extract_words(x_tolerance=1, y_tolerance=3)
    if not words:
        return []
    words = sorted(words, key=lambda w: (round(w['top'],1), w['x0']))
    lines, cur_top, buf = [], None, []
    for w in words:
        if cur_top is None or abs(w['top'] - cur_top) <= 5:
            buf.append(w); cur_top = w['top'] if cur_top is None else cur_top
        else:
            lines.append(" ".join(x['text'] for x in sorted(buf, key=lambda t:t['x0'])))
            buf = [w]; cur_top = w['top']
    if buf:
        lines.append(" ".join(x['text'] for x in sorted(buf, key=lambda t:t['x0'])))
    return [ln.strip() for ln in lines if ln.strip()]

def process_column(lines, out_rows):
    """Parcourt les lignes d'UNE colonne et ajoute les élèves filtrés par option."""
    ecole_actuelle = None
    code_ecole_actuel = None
    option_actuelle = None
    collecting = False  # True si l'option courante est autorisée

    for line in lines:
        # nom d'école
        if ecole_pattern.match(line) and not line[0].isdigit() and not is_header_like(line):
            ecole_actuelle = normalize_school_name(line)  # <-- normalisation ici
            continue

        # code d'école -> fixe l'option + flag collecting
        m = code_ecole_re.search(line)
        if m:
            code_ecole_actuel = m.group(1).strip()
            parts = [p.strip() for p in code_ecole_actuel.split("/") if p.strip()]
            option_actuelle = parts[1][:3] if len(parts) > 1 else None
            collecting = option_actuelle in OPTIONS_ALLOWED
            continue

        # élève
        tokens = line.split()
        if len(tokens) >= 4 and tokens[0].isdigit():
            if tokens[-2].upper() in ("M","F") and percent_re.match(tokens[-1]):
                if not collecting:
                    continue  # <-- ignore les options non souhaitées
                numero = tokens[0]
                # code nettoyé sans espaces ni slashs
                code_clean = re.sub(r"[ /]", "", code_ecole_actuel) if code_ecole_actuel else ""
                numero_concat = f"{code_clean}{numero}" if code_clean else numero
                nom, postnom = tokens[1], tokens[2]
                prenom = " ".join(tokens[3:-2])
                sexe, pourcentage = tokens[-2], tokens[-1]
                out_rows.append([
                    numero_concat, nom, postnom, prenom, sexe, pourcentage,
                    ecole_actuelle, code_ecole_actuel or "",
                    option_actuelle, options_map.get(option_actuelle, ""),
                    "KINSHASA-FUNA", "2023"
                ])

# -------- extraction 3 colonnes / toutes pages --------
eleves = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        w, h = page.width, page.height
        bboxes = [
            (0, 0, w/2.8, h),          # Col 1
            (w/2.8, 0, w/1.59, h),     # Col 2
            (w/1.57, 0, w, h),         # Col 3
        ]
        for bbox in bboxes:
            lines = extract_lines(page, bbox)
            process_column(lines, eleves)

# -------- écriture Excel avec un TABLEAU --------
headers = [
    "ID Élève", "Nom", "Postnom", "Prénom", "Sexe", "Pourcentage",
    "École", "Code École",
    "Code de l'option", "Option",
    "Province Éducationnelle", "Année"
]

os.makedirs(os.path.dirname(xlsx_path), exist_ok=True)

try:
    import pandas as pd
    # DataFrame
    df = pd.DataFrame(eleves, columns=headers)

    # Écriture via xlsxwriter pour créer un vrai tableau
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
        sheet = "Élèves"
        # Écrire les données sans en-têtes, on les mettra via add_table
        df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=1)

        workbook  = writer.book
        worksheet = writer.sheets[sheet]

        rows, cols = df.shape
        col_settings = [{"header": h} for h in headers]

        # Créer le tableau Excel (A1 ... largeur/hauteur dynamiques)
        worksheet.add_table(0, 0, rows, cols-1, {
            "columns": col_settings,
            "style":   "Table Style Medium 9",
            "name":    "Eleves",
        })

        # Largeur de colonnes (optionnel)
        for i, h in enumerate(headers):
            worksheet.set_column(i, i, max(12, min(35, len(h)+4)))

# except Exception:
#     # Fallback openpyxl : créer un tableau via Table()
#     from openpyxl import Workbook
#     from openpyxl.utils import get_column_letter
#     from openpyxl.worksheet.table import Table, TableStyleInfo

#     wb = Workbook()
#     ws = wb.active
#     ws.title = "Élèves"
#     ws.append(headers)
#     for row in eleves:
#         ws.append(row)

#     max_row = ws.max_row
#     max_col = ws.max_column
#     ref = f"A1:{get_column_letter(max_col)}{max_row}"
#     tbl = Table(displayName="Eleves", ref=ref)
#     tbl.tableStyleInfo = TableStyleInfo(
#         name="TableStyleMedium9",
#         showRowStripes=True,
#         showColumnStripes=False
#     )
#     ws.add_table(tbl)
#     wb.save(xlsx_path)

# print(f"✅ Excel (avec Tableau) créé : {xlsx_path}  —  {len(eleves)} lignes")


NameError: name 'aggregats' is not defined

In [1]:
# pip install pdfplumber pandas xlsxwriter openpyxl

import os
import re
import pdfplumber

pdf_path  = r"C:/Users/Famille/OneDrive/Documents/Palmares/2023/Palmares_12_KINSHASA-FUNA.pdf"
xlsx_path = r"C:/Users/Famille/OneDrive/Documents/Palmares/2023/eleves_KINSHASA_FUNA.xlsx"

# Options conservées (on ne change pas ce comportement)
options_map = {
    "101": "LATIN-PHILOSOPHIE",
    "102": "SCIENCES",
    "201": "PEDAGOGIE GENERALE",
    "301": "COMMERCIALE ET GESTION",
}
OPTIONS_ALLOWED = set(options_map.keys())

# --- Normalisation du nom d'école ---
def normalize_school_name(name: str) -> str:
    """
    Règles demandées :
      I. -> INSTITUT
      INST -> INSTITUT
      I.TITUT -> INSTITUT
      CS / C.S / C. S -> COMPLEXE SCOLAIRE
      COL -> COLLEGE
      L. -> LYCEE
      E. -> ECOLE
      ST -> SAINT
      L. T. -> LYCEE TECHNIQUE
      GS -> GROUPE SCOLAIRE
      Rev. -> REVEREND
      ND -> NOTRE DAME
      INST -> INSTITUT
      CM -> COLLEGE MODERNE
      C. S -> COMPLEXE SCOLAIRE
      I. T. -> INSTITUT TECHINIQUE
      CSFRANCO -> COMPLEXE SCOLAIRE FRANCOPHONE
      ST -> SAINT
      STE -> SAINTE
      M. -> MARIE
      JEAN-M. -> JEAN-MARIE
      COL M. -> COLLEGE MARIE
      Pst -> PASTEUR
      PED -> PEDAGOGIQUE
      D'EX -> D'EXCELLENCE
    """
    n = (name or "").upper()

    # ordre important (composés avant simples)
    rules = [
        (r'(?<!\w)L\.\s*T\.?(?!\w)', 'LYCEE TECHNIQUE'),         # L. T.
        (r'(?<!\w)I\.\s*T\.?(?!\w)', 'INSTITUT TECHNIQUE'),         # I. T.
        (r'(?<!\w)REV\.?(?!\w)',       'REVEREND'),               # Rev / Rev.
        (r'(?<!\w)C[.\s]*S[.\s]?(?!\w)','COMPLEXE SCOLAIRE'),     # C.S / C. S / CS / C S
        (r'(?<!\w)CSFRANCO(?!\w)',         'COMPLEXE SCOLAIRE FRANCOPHONE'),
        (r'(?<!\w)I\.(?!\w)',         'INSTITUT'),                # I.
        (r'(?<!\w)INST(?!\w)',        'INSTITUT'),                # INST
        (r'(?<!\w)I.TITUT(?!\w)',        'INSTITUT'),
        (r'(?<!\w)I. SC(?!\w)',        'INSTITUT SCIENTIFIQUE'),
        (r'(?<!\w)GS(?!\w)',          'GROUPE SCOLAIRE'),         # GS
        (r'(?<!\w)COL(?!\w)',         'COLLEGE'),                 # COL
        (r'(?<!\w)CM(?!\w)',          'COLLEGE MODERNE'),         # CM
        (r'(?<!\w)L\.(?!\w)',         'LYCEE'),
        (r'(?<!\w)L\. M.(?!\w)',         'LYCEE MADAME'),         # L. M.
        (r'(?<!\w)E\.(?!\w)',         'ECOLE'),                   # E.
        (r'(?<!\w)ST(?!\w)',          'SAINT'),                   # ST
        (r'(?<!\w)STE(?!\w)',          'SAINTE'),                 # STE
        (r'(?<!\w)ND(?!\w)',          'NOTRE DAME'),              # ND
        (r'(?<!\w)M.(?!\w)',          'MARIE'),                   # MARIE
        (r'(?<!\w)JEAN-M.(?!\w)',          'JEAN-MARIE'),          #JEAN-MARIE
        (r'(?<!\w)COL M.(?!\w)',          'COLLEGE MARIE'),        # MAIRE
        (r'(?<!\w)Pst(?!\w)',        'PASTEUR'),
        (r'(?<!\w)PED(?!\w)',        'PEDAGOGIQUE'),
        (r'(?<!\w)D\'EX(?!\w)', 'D\'EXCELLENCE'),                    # D'EX
    ]
    for pat, rep in rules:
        n = re.sub(pat, rep, n)

    # espaces multiples -> un seul
    n = re.sub(r'\s+', ' ', n).strip()
    return n

# Détection d'un vrai nom d'école (en excluant les entêtes)
ecole_pattern = re.compile(r'^[A-Z0-9\.\s/\-]+$')
def is_header_like(line: str) -> bool:
    l = line.upper().strip()
    return l.startswith(("OPTION", "PROVINCE", "CODE", "PARTICIP", "RÉUSSIT", "REUSSIT"))

percent_re = re.compile(r'^\d+(?:[.,]\d+)?$')
code_ecole_re = re.compile(r'Code\s*:\s*([\d\s/]+)')

def extract_lines(page, bbox):
    """Regroupe les mots en lignes (haut->bas, gauche->droite) dans une bbox."""
    words = page.crop(bbox).extract_words(x_tolerance=1, y_tolerance=3)
    if not words:
        return []
    words = sorted(words, key=lambda w: (round(w['top'],1), w['x0']))
    lines, cur_top, buf = [], None, []
    for w in words:
        if cur_top is None or abs(w['top'] - cur_top) <= 5:
            buf.append(w); cur_top = w['top'] if cur_top is None else cur_top
        else:
            lines.append(" ".join(x['text'] for x in sorted(buf, key=lambda t:t['x0'])))
            buf = [w]; cur_top = w['top']
    if buf:
        lines.append(" ".join(x['text'] for x in sorted(buf, key=lambda t:t['x0'])))
    return [ln.strip() for ln in lines if ln.strip()]

def process_column(lines, out_rows):
    """Parcourt les lignes d'UNE colonne et ajoute les élèves filtrés par option."""
    ecole_actuelle = None
    code_ecole_actuel = None
    option_actuelle = None
    collecting = False  # True si l'option courante est autorisée

    for line in lines:
        # nom d'école
        if ecole_pattern.match(line) and not line[0].isdigit() and not is_header_like(line):
            ecole_actuelle = normalize_school_name(line)  # <-- normalisation ici
            continue

        # code d'école -> fixe l'option + flag collecting
        m = code_ecole_re.search(line)
        if m:
            code_ecole_actuel = m.group(1).strip()
            parts = [p.strip() for p in code_ecole_actuel.split("/") if p.strip()]
            option_actuelle = parts[1][:3] if len(parts) > 1 else None
            collecting = option_actuelle in OPTIONS_ALLOWED
            continue

        # élève
        tokens = line.split()
        if len(tokens) >= 4 and tokens[0].isdigit():
            if tokens[-2].upper() in ("M","F") and percent_re.match(tokens[-1]):
                if not collecting:
                    continue  # <-- ignore les options non souhaitées
                numero = tokens[0]
                # code nettoyé sans espaces ni slashs
                code_clean = re.sub(r"[ /]", "", code_ecole_actuel) if code_ecole_actuel else ""
                numero_concat = f"{code_clean}{numero}" if code_clean else numero
                nom, postnom = tokens[1], tokens[2]
                prenom = " ".join(tokens[3:-2])
                sexe, pourcentage = tokens[-2], tokens[-1]
                out_rows.append([
                    numero_concat, nom, postnom, prenom, sexe, pourcentage,
                    ecole_actuelle, code_ecole_actuel or "",
                    option_actuelle, options_map.get(option_actuelle, ""),
                    "KINSHASA-FUNA", "2023"
                ])

# -------- extraction 3 colonnes / toutes pages --------
eleves = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        w, h = page.width, page.height
        bboxes = [
            (0, 0, w/2.8, h),          # Col 1
            (w/2.8, 0, w/1.59, h),     # Col 2
            (w/1.57, 0, w, h),         # Col 3
        ]
        for bbox in bboxes:
            lines = extract_lines(page, bbox)
            process_column(lines, eleves)

# -------- écriture Excel avec un TABLEAU --------
headers = [
    "ID Élève", "Nom", "Postnom", "Prénom", "Sexe", "Pourcentage",
    "École", "Code École",
    "Code de l'option", "Option",
    "Province Éducationnelle", "Année"
]

os.makedirs(os.path.dirname(xlsx_path), exist_ok=True)

try:
    import pandas as pd
    # DataFrame
    df = pd.DataFrame(eleves, columns=headers)

    # Écriture via xlsxwriter pour créer un vrai tableau
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
        sheet = "Élèves"
        # Écrire les données sans en-têtes, on les mettra via add_table
        df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=1)

        workbook  = writer.book
        worksheet = writer.sheets[sheet]

        rows, cols = df.shape
        col_settings = [{"header": h} for h in headers]

        # Créer le tableau Excel (A1 ... largeur/hauteur dynamiques)
        worksheet.add_table(0, 0, rows, cols-1, {
            "columns": col_settings,
            "style":   "Table Style Medium 9",
            "name":    "Eleves",
        })

        # Largeur de colonnes (optionnel)
        for i, h in enumerate(headers):
            worksheet.set_column(i, i, max(12, min(35, len(h)+4)))

except Exception:
    # Fallback openpyxl : créer un tableau via Table()
    from openpyxl import Workbook
    from openpyxl.utils import get_column_letter
    from openpyxl.worksheet.table import Table, TableStyleInfo

    wb = Workbook()
    ws = wb.active
    ws.title = "Élèves"
    ws.append(headers)
    for row in eleves:
        ws.append(row)

    max_row = ws.max_row
    max_col = ws.max_column
    ref = f"A1:{get_column_letter(max_col)}{max_row}"
    tbl = Table(displayName="Eleves", ref=ref)
    tbl.tableStyleInfo = TableStyleInfo(
        name="TableStyleMedium9",
        showRowStripes=True,
        showColumnStripes=False
    )
    ws.add_table(tbl)
    wb.save(xlsx_path)

print(f"✅ Excel (avec Tableau) créé : {xlsx_path}  —  {len(eleves)} lignes")


✅ Excel (avec Tableau) créé : C:/Users/Famille/OneDrive/Documents/Palmares/2023/eleves_KINSHASA_FUNA.xlsx  —  10903 lignes


In [1]:
# pip install pdfplumber pandas xlsxwriter openpyxl

import os
import re
import pdfplumber

pdf_path  = r"C:/Users/Famille/OneDrive/Documents/Palmares/2023/Palmares_12_KINSHASA-FUNA.pdf"
xlsx_path = r"C:/Users/Famille/OneDrive/Documents/Palmares/2023/aggregats_ecoles_KINSHASA_FUNA.xlsx"

PROVINCE_EDUC = "KINSHASA-FUNA"
ANNEE = "2023"

# --- Normalisation du nom d'école ---
def normalize_school_name(name: str) -> str:
    n = (name or "").upper()
    rules = [
        (r'(?<!\w)L\.\s*T\.?(?!\w)', 'LYCEE TECHNIQUE'),         # L. T.
        (r'(?<!\w)I\.\s*T\.?(?!\w)', 'INSTITUT TECHNIQUE'),      # I. T.
        (r'(?<!\w)REV\.?(?!\w)',       'REVEREND'),               # Rev / Rev.
        (r'(?<!\w)C[.\s]*S[.\s]?(?!\w)','COMPLEXE SCOLAIRE'),     # C.S / C. S / CS / C S
        (r'(?<!\w)CSFRANCO(?!\w)',     'COMPLEXE SCOLAIRE FRANCOPHONE'),
        (r'(?<!\w)I\.(?!\w)',         'INSTITUT'),                # I.
        (r'(?<!\w)INST(?!\w)',        'INSTITUT'),                # INST
        (r'(?<!\w)I\.?TITUT(?!\w)',   'INSTITUT'),                # I.TITUT
        (r'(?<!\w)I\.\s*SC(?!\w)',    'INSTITUT SCIENTIFIQUE'),   # I. SC
        (r'(?<!\w)GS(?!\w)',          'GROUPE SCOLAIRE'),         # GS
        (r'(?<!\w)COL(?!\w)',         'COLLEGE'),                 # COL
        (r'(?<!\w)COLL.(?!\w)',       'COLLEGE'),
        (r'(?<!\w)CM(?!\w)',          'COLLEGE MODERNE'),         # CM
        (r'(?<!\w)L\.(?!\w)',         'LYCEE'),
        (r'(?<!\w)L\.\s*M\.(?!\w)',   'LYCEE MADAME'),            # L. M.
        (r'(?<!\w)E\.(?!\w)',         'ECOLE'),                   # E.
        (r'(?<!\w)ST(?!\w)',          'SAINT'),                   # ST
        (r'(?<!\w)STE(?!\w)',         'SAINTE'),                  # STE
        (r'(?<!\w)ND(?!\w)',          'NOTRE DAME'),              # ND
        (r'(?<!\w)M\.(?!\w)',         'MARIE'),                   # M.
        (r'(?<!\w)JEAN-M\.(?!\w)',    'JEAN-MARIE'),
        (r'(?<!\w)COL\s*M\.(?!\w)',   'COLLEGE MARIE'),
        (r'(?<!\w)Pst(?!\w)',         'PASTEUR'),
        (r'(?<!\w)PED(?!\w)',         'PEDAGOGIQUE'),
        (r'(?<!\w)D\'EX(?!\w)',       'D\'EXCELLENCE'),           # D'EX
        (r'(?<!\w)Prof(?!\w)',        'PROFESSEUR'),
    ]
    for pat, rep in rules:
        n = re.sub(pat, rep, n)
    return re.sub(r'\s+', ' ', n).strip()

# --- Helpers extraction lignes ---
ecole_pattern = re.compile(r'^[A-Z0-9\.\s/\-]+$')
def is_header_like(line: str) -> bool:
    l = (line or "").upper().strip()
    return l.startswith(("OPTION", "PROVINCE", "CODE", "PARTICIP", "RÉUSSIT", "REUSSIT"))

# Code sur la ligne (strict pour éviter d'englober le début d'un élève)
RE_CODE_LINE = re.compile(r"Code\s*:\s*([0-9]+(?:\s*/\s*[0-9]+){1,})", re.I)
RE_PART_ANY  = re.compile(r"^Participants\b.*", re.I)
RE_REUS_ANY  = re.compile(r"^(Réussites|Reussites)\b.*", re.I)

def clean_code_keep_digits(raw: str) -> str:
    # CodeEcole affiché: seulement chiffres (sans espaces ni '/')
    return re.sub(r"\D+", "", raw or "")

def extract_lines(page, bbox, x_tol=2, y_tol_words=4, y_tol_lines=6):
    words = page.crop(bbox).extract_words(x_tolerance=x_tol, y_tolerance=y_tol_words)
    if not words:
        return []
    words = sorted(words, key=lambda w: (round(w['top'],1), w['x0']))
    lines, cur_top, buf = [], None, []
    for w in words:
        if cur_top is None or abs(w['top'] - cur_top) <= y_tol_lines:
            buf.append(w); cur_top = w['top'] if cur_top is None else cur_top
        else:
            lines.append(" ".join(x['text'] for x in sorted(buf, key=lambda t:t['x0'])))
            buf = [w]; cur_top = w['top']
    if buf:
        lines.append(" ".join(x['text'] for x in sorted(buf, key=lambda t:t['x0'])))
    return [re.sub(r"\s+", " ", ln.replace("\xa0"," ")).strip() for ln in lines if ln.strip()]

def linearize(pdf):
    """Liste globale (page, col, line_idx, text) dans l'ordre de lecture 3 colonnes."""
    seq = []
    for p_idx, page in enumerate(pdf.pages):
        w, h = page.width, page.height
        bboxes = [
            (0, 0, w/2.8, h),          # col 1
            (w/2.8, 0, w/1.59, h),     # col 2
            (w/1.57, 0, w, h),         # col 3
        ]
        for c_idx, bbox in enumerate(bboxes):
            lines = extract_lines(page, bbox)
            for l_idx, text in enumerate(lines):
                seq.append((p_idx, c_idx, l_idx, text))
    return seq

def find_school_name(seq, idx):
    """Nom d'école à proximité: d'abord en arrière (3 lignes), sinon en avant (3 lignes)."""
    for k in range(idx-1, max(-1, idx-4), -1):
        if k < 0: break
        t = seq[k][3]
        if t and not t[0].isdigit() and ecole_pattern.match(t) and not is_header_like(t):
            return normalize_school_name(t)
    for k in range(idx+1, min(idx+4, len(seq))):
        t = seq[k][3]
        if t and not t[0].isdigit() and ecole_pattern.match(t) and not is_header_like(t):
            return normalize_school_name(t)
    return ""

def parse_number_block(seq, start_idx, label_regex):
    """
    Cherche Participants / Réussites en avant même si c'est dans la colonne/page suivante.
    total = 1er entier ; femmes = 1er 'Dont ... F' si présent.
    Stop si nouveau Code: ou après ~30 lignes.
    """
    limit = min(len(seq), start_idx + 30)
    for k in range(start_idx, limit):
        line = seq[k][3]
        if RE_CODE_LINE.search(line):
            return None, None
        if label_regex.match(line):
            m_total = re.search(r"(\d+)", line)
            m_f     = re.search(r"Dont\s*[:\uFF1A]?\s*(\d+)\s*F\.?\b", line, re.I)
            if not m_total and k+1 < len(seq):
                nxt = seq[k+1][3]
                m_total = re.search(r"(\d+)", nxt)
                m_f     = re.search(r"Dont\s*[:\uFF1A]?\s*(\d+)\s*F\.?\b", nxt, re.I)
            if m_total:
                tot = int(m_total.group(1))
                fem = int(m_f.group(1)) if m_f else 0
                return tot, fem
    return None, None

# -------- Construction de la carte des écoles (agrégats) --------
def build_school_aggregates(pdf):
    seq = linearize(pdf)
    school_map = {}  # key: CodeEcole (digits only) -> dict infos
    i = 0
    while i < len(seq):
        text = seq[i][3]
        m = RE_CODE_LINE.search(text)
        if m:
            raw_code = m.group(1).strip()
            code_clean = clean_code_keep_digits(raw_code)
            ecole = find_school_name(seq, i)

            p_tot, p_f = parse_number_block(seq, i+1, RE_PART_ANY)
            r_tot, r_f = parse_number_block(seq, i+1, RE_REUS_ANY)

            p_tot = p_tot or 0; p_f = p_f or 0; p_m = max(p_tot - p_f, 0)
            r_tot = r_tot or 0; r_f = r_f or 0; r_m = max(r_tot - r_f, 0)

            school_map[code_clean] = {
                "CodeEcole": code_clean,
                "Ecole": ecole,
                "ProvinceEduc": PROVINCE_EDUC,
                "Annee": ANNEE,
                "PartTot": p_tot, "PartH": p_m, "PartF": p_f,
                "ReussTot": r_tot, "ReussH": r_m, "ReussF": r_f,
            }
        i += 1
    return list(school_map.values())

# ==================== EXÉCUTION ====================
aggregats = []
with pdfplumber.open(pdf_path) as pdf:
    aggregats = build_school_aggregates(pdf)

# -------- écriture Excel avec un TABLEAU --------
headers = [
    "CodeEcole", "Ecole", "ProvinceEduc", "Annee",
    "PartTot", "PartH", "PartF",
    "ReussTot", "ReussH", "ReussF"
]

os.makedirs(os.path.dirname(xlsx_path), exist_ok=True)

try:
    import pandas as pd
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
        sheet = "Écoles"
        import pandas as pd
        df = pd.DataFrame(aggregats, columns=headers)
        df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=1)

        workbook  = writer.book
        worksheet = writer.sheets[sheet]
        rows, cols = df.shape
        col_settings = [{"header": h} for h in headers]
        worksheet.add_table(0, 0, rows, cols-1, {
            "columns": col_settings,
            "style":   "Table Style Medium 9",
            "name":    "AggregatsEcoles",
        })
        for i, h in enumerate(headers):
            worksheet.set_column(i, i, max(12, min(40, len(h)+6)))
except Exception:
    from openpyxl import Workbook
    from openpyxl.utils import get_column_letter
    from openpyxl.worksheet.table import Table, TableStyleInfo
    wb = Workbook(); ws = wb.active; ws.title = "Écoles"
    ws.append(headers)
    for row in aggregats:
        ws.append([row.get(h, "") for h in headers])
    max_row = ws.max_row; max_col = ws.max_column
    ref = f"A1:{get_column_letter(max_col)}{max_row}"
    tbl = Table(displayName="AggregatsEcoles", ref=ref)
    tbl.tableStyleInfo = TableStyleInfo(
        name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False
    )
    ws.add_table(tbl); wb.save(xlsx_path)

print(f"✅ Excel (Tableau) créé : {xlsx_path} — {len(aggregats)} écoles")


✅ Excel (Tableau) créé : C:/Users/Famille/OneDrive/Documents/Palmares/2023/aggregats_ecoles_KINSHASA_FUNA.xlsx — 1286 écoles
